# Biostat 257 Homework 5

**Due June 8 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, westmere)


Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW4, we used the nonlinear programming (NLP) approach (Newton type algorithms) for optimization. In this assignment, we derive and implement an expectation-maximization (EM) algorithm for the same problem.

In [2]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random, Revise

## Q1. (10 pts) Refresher on normal-normal model

Assume the conditional distribution
$$
\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)
$$
and the prior distribution
$$
\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma}).
$$
By the Bayes theorem, the posterior distribution is
\begin{eqnarray*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})}, \end{eqnarray*}
where $f$ denotes corresponding density. 

Show that the posterior distribution of random effects $\boldsymbol{\gamma}$ is a multivariate normal with mean
\begin{eqnarray*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{eqnarray*}
and covariance
\begin{eqnarray*}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&=& \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{eqnarray*}

## Sol Q1
### Method 1: $Mm$-formula
Re-write this as a hierachical Bayesian linear model:
\begin{align}
\mathbf{y}-\mathbf{X\beta} &=\mathbf{Z}\gamma +e_y\\
e_y&\sim\mathcal{N}(0,\sigma^2I)\\
\gamma&\sim\mathcal{N}(0,\Sigma)
\end{align}
Then by Dr. Banerjee's $Mm$-formula in *Biostat 250C*, we have
$$p(\gamma|y)=\mathcal{N}(\gamma|Mm,M)$$
where
\begin{align}
M^{-1}&=Z^TV_y^{-1}Z+V_{\gamma}^{-1}=\frac{\mathbf{Z}^T\mathbf{Z}}{\sigma^2}+\Sigma^{-1}\\
m&=V_\gamma^{-1}\mu_\gamma+Z^TV_y^{-1}(y-\mu_y)=\frac{1}{\sigma^2}Z^T(y-X\beta)
\end{align}
Thus,
\begin{align}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=Mm= \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})\\
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=M= (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1}
\end{align}

### Method 2: conditional multivariate normal distribution

Define $W=\mathbf{y}-\mathbf{X\beta}$, then re-write the above as an augmented linear model:
\begin{align}
W &=\mathbf{Z}e_\gamma +e_W\\
\gamma&=e_\gamma\\
e_W&\sim\mathcal{N}(0,\sigma^2I)\\
e_\gamma&\sim\mathcal{N}(0,\Sigma)
\end{align}

Then immediately we have
$$\left[\begin{matrix}W\\\gamma\end{matrix}\right]\sim\mathcal{N}\left(\left[\begin{matrix} 0\\0\end{matrix}\right],\left[\begin{matrix}Z\Sigma Z^T+\sigma^2I& Z\Sigma\\\Sigma Z^T &\Sigma \end{matrix}\right]\right)$$

Thus, by conditional multivariate normal formula in *Biostat 200C* and *Biostat 250A*, we have

$$\gamma|W\sim\mathcal{N}(\mu_{\gamma|W},\Sigma_{\gamma|W})$$

where
\begin{align}
\mu_{\gamma|W}&=\Sigma Z^T\left(Z\Sigma Z^T+\sigma^2I\right)^{-1}W\\
\Sigma_{\gamma|W}&=\Sigma-\Sigma Z^T(Z\Sigma Z^T+\sigma^2I)^{-1}Z\Sigma
\end{align}

Subsituting $W$ with $y-X\beta$ will reproduce Dr. Zhou's solution.

### Method 3: Completing the square

This method is identical to the `Mm-formula` above, so I shall omit it.

## Q2. (20 pts) Derive EM algorithm

1. Write down the complete log-likelihood
$$
\sum_{i=1}^m \log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
$$

2. Derive the $Q$ function (E-step).
$$
Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}).
$$

3. Derive the EM (or ECM) update of $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$ (M-step). 

## Sol Q2

#### Q2.1

- By Q1, the complete likelihood for a single datum is indeed
\begin{align}
\exp\left(\mathcal{L}_i(\beta,\Sigma,\sigma^2)\right)&=\frac{1}{(2\pi)^{n_i/2}\sigma^{n_i}}\exp\{-\frac{1}{2\sigma^2}\lVert y_i-X_i\beta-Z_i\gamma_i \lVert_2^2\}\times\frac{1}{(2\pi)^{q/2}|\Sigma|^{1/2}}\exp\{-\frac{1}{2}\gamma_i^T\Sigma^{-1}\gamma_i\}\\
&=\frac{1}{(2\pi)^{\frac{q+n_i}{2}}(\sigma^2)^{\frac{n_i}{2}}|\Sigma|^{\frac{1}{2}}}\exp\left\{-\frac{1}{2\sigma^2}\left(\lVert y_i-X_i\beta-Z_i\gamma_i \lVert_2^2\ +\ \sigma^2\gamma_i^T\Sigma^{-1}\gamma_i\right)\right\}
\end{align}

Hence, the log-likelihood is

$$\mathcal{L}_i(\beta,\Sigma,\sigma^2)=-\left[\frac{q+n_i}{2}\log2\pi+\frac{n_i}{2}\log\sigma^2+\frac{1}{2}\log|\Sigma|\right]-\frac{1}{2\sigma^2}\left[\lVert y_i-X_i\beta-Z_i\gamma_i\lVert_2^2+\sigma^2\gamma_i^T\Sigma^{-1}\gamma_i\right]$$

Therefore, the complete log-likelihood for the whole dataset is 
\begin{align}
\mathcal{L}(\beta,\Sigma,\sigma^2)&=\sum_{i=1}^m\mathcal{L}_i(\beta,\Sigma,\sigma^2)\\
&=-\left[\frac{mq+n}{2}\log2\pi+\frac{n}{2}\log\sigma^2+\frac{m}{2}\log|\Sigma|\right] - \frac{1}{2\sigma^2}\left[\sum_{i=1}^m\lVert y_i-X_i\beta-Z_i\gamma_i\lVert_2^2+\sigma^2\sum_{i=1}^m\gamma_i^T\Sigma^{-1}\gamma_i\right]
\end{align}

#### Q2.2
First, note that
$$\gamma_i^TZ_i^TZ_i\gamma_i=\text{Tr}(\gamma_i\gamma_i^TZ_i^TZ_i)$$
$$\gamma_i^T\Sigma^{-1}\gamma_i=\text{Tr}(\gamma_i\gamma_i^T\Sigma^{-1})$$

Let $w_i=y_i-X_i\beta^{(t)}$, then by Q1, we know that the first and second **geometric moments** are

\begin{align}
\mu_{1}^{i}=\mathbb{E}\left(\gamma_i\mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}\right)&=\sigma^{-2(t)}\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}Z_i^Tw_i\\
\mu_{2}^{i}=\mathbb{E}\left(\gamma_i\gamma_i^T\mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}\right)&=\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}+\sigma^{-4(t)}\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}Z_i^Tw_iw_i^TZ_i\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}
\end{align}

Hence, the $\mathcal{Q}$ function is

$$\mathcal{Q}=C-\frac{n}{2}\log\sigma^2-\frac{m}{2}\log|\Sigma|+\frac{1}{2\sigma^2}\left[\sum_{i=1}^m\lVert w_i\lVert_2^2+\sigma^2\sum_{i=1}^m\text{Tr}\left(\mu_2^i \left(\frac{Z_i^TZ_i}{\sigma^2}+\Sigma^{-1}\right)\right)-2\sum_{i=1}^mw_i^TZ_i\mu_1^i\right]$$

where $C$ is a constant independent of parameters.

#### Q2.3

Take derivatives w.r.t. all parameters and set them to 0,

\begin{align}
\widehat{\sigma^2}^{(t+1)}&=\frac{1}{n}\sum_{i=1}^m\left[\lVert\widehat{w}_i\lVert_2^2+\text{Tr}(\mu_2^iZ_i^TZ_i)-2w_i^TZ_i\mu_1^i\right]\\
\widehat{\beta}^{(t+1)}
&=\left(\sum_{i=1}^mX_i^TX_i\right)^{-1}\left(\sum_{i=1}^mX_i^T(y_i-Z_i\mathbf{\mu}_1^{i})\right)\\
\widehat{\Sigma}^{(t+1)}&=\frac{1}{m}\left(\sum_{i=1}^m\mu_2^i\right)
\end{align}

Again, I took advantge of Dr. Zhou's [216 notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) for deriving derivatives.

## Q3. (20 pts) Objective of a single datum

We modify the code from HW4 to evaluate the objective, the conditional mean of $\boldsymbol{\gamma}$, and the conditional variance of $\boldsymbol{\gamma}$. Start-up code is provided below. You do _not_ have to use this code.

### Supplementary materials for Q3

By Q2, we have indeed
\begin{align}
V_i^{(t)}&=\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}\\
\mu^{(t)}_i&=\frac{1}{\sigma^{2(t)}}\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}Z_i^Tw_i
\end{align}

In [3]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ztzu       :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2:: Matrix{T}
    storage_qq3:: Matrix{T}
    storage_I  :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ztzu       = Vector{T}(undef, q)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    storage_qq2= similar(ztz)
    storage_qq3= similar(ztz)
    storage_I  = 1.0 * Matrix(I, q, q)
    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ztzu, ltztr, xtr,
        storage_p, storage_q, 
        xtx, ztx, ztz, ltztzl, storage_qq, storage_qq2, storage_qq3, storage_I)
end

"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # TODO: Evaluate posterior mean and variance
    ######################################    
    # TODO
    
    ## νγ = σ² L(inv(M)inv(Mt))Lt
    #obs.storage_qq2 .= L
    #BLAS.trsm!('R', 'U', 'N', 'N', T(1.0), obs.storage_qq, obs.storage_qq2)
    #mul!(obs.storage_I, obs.storage_qq2, transpose(obs.storage_qq2))
    #axpby!(σ², obs.storage_I, 0.0, obs.νγ)
    BLAS.trsm!('L', 'L', 'N', 'N', 1.0, L, copy!(obs.νγ, obs.storage_I))
    BLAS.trsm!('L', 'L', 'T', 'N', 1.0, L, obs.νγ)
    axpby!(σ²inv, obs.ztz, 1.0, obs.νγ)
    ### Cholesky of (inv(Σ) + ztz/σ²)
    LAPACK.potrf!('L', obs.νγ)
    ### Inverse of (inv(Σ) + ztz/σ²)
    LAPACK.potri!('L', obs.νγ)
    LinearAlgebra.copytri!(obs.νγ, 'L')
    
    ## inv(σ²) (νγ ztr)
    mul!(obs.μγ, obs.νγ, obs.ztr, σ²inv, 0.0)
    
    ###################
    # Return
    ###################        
    return logl
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum in HW2 and HW4.

In [4]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [5]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3261.91775591876
obs.μγ = [-1.2467648280833785, -0.02448852130995821, -0.7973392260495256]
obs.νγ = [0.0007494365950329128 1.9689775091534487e-6 1.6361329929107185e-6; 1.9689775091534487e-6 0.0007323744614646161 -1.0428608787480586e-5; 1.6361329929107185e-6 -1.0428608787480586e-5 0.0007326509694024718]


You will lose all 20 points if following statement throws `AssertionError`.

In [6]:
@assert abs(logl - (-3261.9177559187597)) < 1e-8
@assert norm(obs.μγ - [-1.2467648280832386, 
        -0.02448852130995758, -0.7973392260495442]) < 1e-8
@assert norm(obs.νγ - [0.000749436595166797 1.9689775630068264e-6 1.6361330590481234e-6; 
        1.96897748759551e-6 0.0007323744616953198 -1.0428608786771824e-5; 
        1.6361330146430107e-6 -1.0428608802351851e-5 0.0007326509694599695]) < 1e-8

### Efficiency

Benchmark for efficiency.

In [7]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.304 μs (0.00% GC)
  median time:      1.367 μs (0.00% GC)
  mean time:        1.384 μs (0.00% GC)
  maximum time:     28.950 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

My median runt time is 1.7μs. You will get full credit if the median run time is within 10μs. The points you will get are

In [8]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0

In [9]:
# # check for type stability
# @code_warntype logl!(obs, β, Σ, L, σ²)

In [10]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, Σ, L, σ²); end
# Profile.print(format=:flat)

## Q4. LmmModel type

We modify the `LmmModel` type in HW4 to hold all data points, model parameters, and intermediate arrays.

In [11]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat}
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β      :: Vector{T}
    Σ      :: Matrix{T}
    L      :: Matrix{T}
    σ²     :: Vector{T}    
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty    :: Vector{T}
    xtzu   :: Vector{T}
    ztu    :: Vector{T}
    xtr    :: Vector{T}
    ztr2   :: Vector{T}
    xtxinv :: Matrix{T}
    ztz2   :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    n      = size(obsvec[1].X, 1)
    p      = size(obsvec[1].X, 2)
    q      = size(obsvec[1].Z, 2)
    # parameters
    β      = Vector{T}(undef, p)
    Σ      = Matrix{T}(undef, q, q)
    L      = Matrix{T}(undef, q, q)
    σ²     = Vector{T}(undef, 1)    
    # intermediate arrays
    xty    = zeros(T, p)
    xtzu   = similar(xty)
    ztu    = Vector{T}(undef, n)
    xtr    = similar(xty)
    ztr2   = Vector{T}(undef, abs2(q))
    xtxinv = zeros(T, p, p)
    # pre-calculate \sum_i Xi^T Xi and \sum_i Xi^T y_i
    @inbounds for i in eachindex(obsvec)
        obs = obsvec[i]
        BLAS.axpy!(T(1), obs.xtx, xtxinv)
        BLAS.axpy!(T(1), obs.xty, xty)
    end
    # invert X'X
    LAPACK.potrf!('U', xtxinv)
    LAPACK.potri!('U', xtxinv)
    LinearAlgebra.copytri!(xtxinv, 'U')
    ztz2   = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, Σ, L, σ², xty, xtzu, ztu, xtr, ztr2, xtxinv, ztz2)
end

LmmModel

## Q5. Implement EM update

Let's write the key function `update_em!` that performs one iteration of EM update.

### Supplementary materials for Q5

The EM algorithm is

\begin{align}
\widehat{\sigma^2}^{(t+1)}&=\frac{1}{n}\sum_{i=1}^m\left[\lVert\widehat{w}_i\lVert_2^2+\text{Tr}(\mu_2^iZ_i^TZ_i)-2w_i^TZ_i\mu_1^i\right]\\
\widehat{\beta}^{(t+1)}
&=\left(\sum_{i=1}^mX_i^TX_i\right)^{-1}\mathbf{X}^T\left(\mathbf{y}-\mathbf{Z}\mathbf{\mu}_1\right)\\
&=\left(\sum_{i=1}^mX_i^TX_i\right)^{-1}\left(\sum_{i=1}^mX_i^T(y_i-Z_i\mathbf{\mu}_1^{i})\right)\\
\widehat{\Sigma}^{(t+1)}&=\frac{1}{m}\left(\sum_{i=1}^m\mu_2^i\right)
\end{align}

where

\begin{align}
\mu_{1}^{i}=\mathbb{E}\left(\gamma_i\mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}\right)&=\sigma^{-2(t)}\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}Z_i^Tw_i\\
\mu_{2}^{i}=\mathbb{E}\left(\gamma_i\gamma_i^T\mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}\right)&=\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}+\sigma^{-4(t)}\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}Z_i^Tw_iw_i^TZ_i\left(\frac{Z_i^TZ_i}{\sigma^{2(t)}}+(\boldsymbol{\Sigma}^{(t)})^{-1}\right)^{-1}
\end{align}

In [12]:
"""
    update_em!(m::LmmModel, updater::Bool = false)

Perform one iteration of EM update. It returns the log-likelihood calculated 
from input `m.β`, `m.Σ`, `m.L`, and `m.σ²`. These fields are then overwritten 
by the next EM iterate. The fields `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` are updated according to the resultant `m.β`. If `updater==true`, 
the function first updates `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` according to `m.β`. If `updater==false`, it assumes these fields 
are pre-computed.
"""
function update_em!(m::LmmModel{T}, updater::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    M = length(m.data)
    # TODO: update m.β
    fill!(m.xtzu, 0.0)
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.Σ, m.L, m.σ²[1], true)
        
        mul!(obs.storage_p, transpose(obs.ztx), obs.μγ)
        BLAS.axpby!(T(1), obs.storage_p, T(1), m.xtzu)
    end
    axpby!(1.0, m.xty, -1.0, m.xtzu)
    BLAS.gemv!('N', 1.0, m.xtxinv, m.xtzu, 0.0, m.β)
    
    # TODO: update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
    sum_n = 0.0
    fill!(m.σ², 0.0)
    fill!(m.Σ, 0.0)
    
    @inbounds for i in 1:M
        obs = m.data[i]
        
        if updater
        # update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
        # Zt * res
            BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Xt * res = Xt * y - Xt * X * β
            BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # l2 norm of residual vector
            obs.rtr[1] = obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        end
        
        # update m.σ²
        sum_n += size(obs.y)[1]
        m.σ²[1] += obs.rtr[1]
        m.σ²[1] -= 2 * dot(obs.zty, obs.μγ)
        m.σ²[1] += 2 * dot(m.β, obs.storage_p)
        mul!(obs.storage_qq3, obs.νγ, obs.ztz)
        m.σ²[1] += tr(obs.storage_qq3)
        BLAS.gemv!('N', T(1), obs.ztz, obs.μγ, T(0), obs.storage_q)
        m.σ²[1] += dot(obs.μγ, obs.storage_q)
        
        # update m.Σ 
        axpby!(1.0, obs.νγ, 1.0, m.Σ)
        BLAS.ger!(1.0, obs.μγ, obs.μγ, m.Σ)
        
    end
    
    # TODO: update m.σ²
    m.σ²[1] /= sum_n
    
    # update m.L
    lmul!(1/M, m.Σ)
    LAPACK.potrf!('L', copy!(m.L, m.Σ))
    
    # return log-likelihood at input parameter values
    logl
end

update_em!

## Q6. (30 pts) Test data

Let's generate a fake longitudinal data set (same as HW4) to test our algorithm.

In [13]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

### Correctness

Evaluate log-likelihood and gradient at the true parameter values.

In [14]:
copy!(lmm.β, βtrue)
copy!(lmm.Σ, Σtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj1 = update_em!(lmm, true)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²
println()
@show obj2 = update_em!(lmm, false)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²;

obj1 = update_em!(lmm, true) = -2.8342752296337825e6
lmm.β = [0.10001001012634447, 6.498340323625655, -3.50051183240116, 1.001818786999053, 5.001950303046151]
lmm.Σ = [2.069069911252674 0.05260748700681633 0.033127536963117384; 0.05260748700681633 1.121914171088475 0.05589997696166219; 0.033127536963117384 0.05589997696166219 1.012038870789779]
lmm.L = [1.4384261924939612 0.05260748700681633 0.033127536963117384; 0.036572948463629414 1.0585729027937338 0.05589997696166219; 0.0230304044350586 0.052011237980734905 1.0043920063327108]
lmm.σ² = [1.5023480060584489]

obj2 = update_em!(lmm, false) = -2.8342646397564034e6
lmm.β = [0.1000199603628697, 6.49833729110276, -3.5005130517068554, 1.0018219537039756, 5.001953708893931]
lmm.Σ = [2.069122104327052 0.05267124986368272 0.03317503527874422; 0.05267124986368272 1.121801522838958 0.05599535523251236; 0.03317503527874422 0.05599535523251236 1.0120528110994926]
lmm.L = [1.438444334803072 0.05267124986368272 0.03317503527874422; 0.0366168148389

Test correctness. You will loss all 30 points if following code throws `AssertError`.

In [15]:
@assert abs(obj1 - (-2.8342752296337797e6)) < 1e-6
@assert abs(obj2 - (-2.8342646397564034e6)) < 1e-6

### Efficiency

Test efficiency of EM update.

In [16]:
bm_emupdate = @benchmark update_em!($lmm, true) setup=(
    copy!(lmm.β, βtrue);
    copy!(lmm.Σ, Σtrue);
    copy!(lmm.L, Ltrue);
    lmm.σ²[1] = σ²true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.639 ms (0.00% GC)
  median time:      1.661 ms (0.00% GC)
  mean time:        1.686 ms (0.00% GC)
  maximum time:     2.287 ms (0.00% GC)
  --------------
  samples:          2960
  evals/sample:     1

My median run time is 2.17ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [17]:
clamp(10 / (median(bm_emupdate).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [18]:
clamp(10 - median(bm_emupdate).memory / 100, 0, 10)

10.0

## Q7. Starting point

We use the same least squares estimates as in HW4 as starting point. 

In [19]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # LS estimate for β
    mul!(m.β, m.xtxinv, m.xty)
    # LS etimate for σ2 and Σ
    rss, ntotal = zero(T), 0
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)    
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        ntotal += length(obs.y)
        # update Xt * res
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # rss of i-th individual
        rss += obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        # update Zi' * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Zi'Zi ⊗ Zi'Zi
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2)
        # Zi'res ⊗ Zi'res
        kron_axpy!(obs.ztr, obs.ztr, m.ztr2)
    end
    m.σ²[1] = rss / ntotal
    # LS estimate for Σ = LLt
    LAPACK.potrf!('U', m.ztz2)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2)
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2)
    copyto!(m.Σ, m.ztr2)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    for j in 2:q, i in 1:j-1
        m.L[i, j] = 0
    end
    m
end

"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [20]:
init_ls!(lmm)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

lmm.β = [0.12814248060847547, 6.497945035096694, -3.5021387747701356, 1.0026609224782135, 5.002697971721907]
lmm.Σ = [2.079622415940933 0.05523492575567487 0.04021508338255693; 0.055234925755674874 1.113731475210463 0.06543803114908463; 0.04021508338255692 0.06543803114908461 1.0074558313277846]
lmm.L = [1.4420896005245072 0.0 0.0; 0.03830200684866266 1.054639479387069 0.0; 0.027886674564416913 0.0610349951874949 1.0014753586943541]
lmm.σ² = [5.7017098859594215]


1-element Vector{Float64}:
 5.7017098859594215

## Q8. Estimation by EM

We write a function `fit!` that implements the EM algorithm for estimating LMM.

In [21]:
"""
    fit!(m::LmmModel)

Fit an `LmmModel` object by MLE using a EM algorithm. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m       :: LmmModel;
        maxiter :: Integer       = 10_000,
        ftolrel :: AbstractFloat = 1e-12,
        prtfreq :: Integer       = 0
    )
    obj = update_em!(m, true)
    for iter in 0:maxiter
        obj_old = obj
        # EM update
        obj = update_em!(m, false)
        # print obj
        prtfreq > 0 && rem(iter, prtfreq) == 0 && println("iter=$iter, obj=$obj")
        # check monotonicity
        obj < obj_old && (@warn "monotonicity violated")
        # check convergence criterion
        (obj - obj_old) < ftolrel * (abs(obj_old) + 1) && break
        # warning about non-convergence
        iter == maxiter && (@warn "maximum iterations reached")
    end
    m
end


fit!

## Q9. (20 pts) Test drive

Now we can run our EM algorithm to compute the MLE.

In [22]:
# initialize from least squares
init_ls!(lmm)

@time fit!(lmm, prtfreq = 100);

println("objective value at solution: ", update_em!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)

iter=0, obj=-2.8342745309956353e6
iter=100, obj=-2.8342645064545856e6
iter=200, obj=-2.834264506126482e6
iter=300, obj=-2.8342645058246725e6
  0.551527 seconds (151.31 k allocations: 8.524 MiB, 4.81% compilation time)
objective value at solution: -2.8342645057474184e6

solution values:
lmm.β = [0.12758512267178201, 6.498337285661687, -3.500513054241839, 1.001821957286909, 5.001953714469486]
lmm.σ² = [1.5023520664906653]
lmm.L * transpose(lmm.L) = [2.072478822484914 0.11131049762675521 0.06822143422819707; 0.11131049762675521 1.124936863194401 0.11223751393351014; 0.06822143422819707 0.11223751393351014 1.0120528546606307]


3×3 Matrix{Float64}:
 2.07248    0.11131   0.0682214
 0.11131    1.12494   0.112238
 0.0682214  0.112238  1.01205

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [23]:
# objective at solution should be close enough to the optimal
@assert update_em!(lmm) > -2.83426451e6

### Efficiency

My median run time 1.8s. You get 10 points if your median run time is within 5s.

In [24]:
bm_em = @benchmark fit!($lmm) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     517.284 ms (0.00% GC)
  median time:      524.102 ms (0.00% GC)
  mean time:        524.764 ms (0.00% GC)
  maximum time:     532.179 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1

In [25]:
# this is the points you get
clamp(5 / (median(bm_em).time / 1e9) * 10, 0, 10)

10.0

## Q10. (10 pts) EM vs Newton type algorithms

Contrast EM algorithm to the Newton type algorithms (gradient free, gradient based, using Hessian) in HW4, in terms of the stability, convergence rate (how fast the algorithm is converging),  final objective value, total run time, derivation, and implementation efforts. 

| Items | EM algorithm | Newton-type algorithms |
|---|---|---|
| stability | Monotonicity (always increasing) |Not guaranteed to be increasing|
| Convergence rate | Linear (associated with the eigenvalues of Fisher information matrix) | Fast (e.g., second order for Newton-Raphson) |
| Final objective value | -2.83426e6 | -2.83426e6 |
| Total run time | 524.102 ms | 84.246 ms |
| Derivation| CRAZY and PAINFUL!| Matured technology (according to Dr. Zhou and Dr. Vandenberghe)|
|Implementation efforts| Easy (just a few lines)| Extremelly *PAINFUL* to get gradients and Hessian|